# Section 2 Risk Factor Extraction

In section 1, we have downloaded the fincancial reports from 2019Q1 to 2023Q3. In this section, we are going to extract the risk factor text from each of these files. According to the requirement of the 10-K form given by SEC (https://www.sec.gov/files/form10-k.pdf), there is an section named "Item 1A. Risk Factors" included in the form  which shows the potential risks faced by the company. Therefore, it is important for us to filter and cleanse the text from this part so that we can use it for further analysis.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re
import html
import warnings
warnings.filterwarnings("ignore")

# 2.4 Life Sciences Industry (2834, 2836)

In this section, we only focus on companies in Life Sciences Industry.

## 2.4.1 Industry 2834

### 1. Read in the files and store in a dictionary

In [2]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Life Sciences Industry/Industry_2834/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

In [3]:
filePair_dict.keys()

dict_keys(['0000014272-23-000046.txt', '0000806592-22-000024.txt', '0000831547-23-000007.txt', '0000849146-22-000058.txt', '0000868278-23-000004.txt', '0000950170-22-010852.txt', '0000950170-22-011842.txt', '0000950170-22-012521.txt', '0000950170-22-012532.txt', '0000950170-22-018236.txt', '0000950170-22-018865.txt', '0000950170-23-010380.txt', '0000950170-23-010428.txt', '0000950170-23-010432.txt', '0000950170-23-010435.txt', '0000950170-23-010599.txt', '0000950170-23-010603.txt', '0000950170-23-010613.txt', '0000950170-23-010627.txt', '0000950170-23-010757.txt', '0000950170-23-010920.txt', '0000950170-23-010921.txt', '0000950170-23-010967.txt', '0000950170-23-010973.txt', '0000950170-23-010977.txt', '0000950170-23-011000.txt', '0000950170-23-011011.txt', '0000950170-23-011047.txt', '0000950170-23-011138.txt', '0000950170-23-011139.txt', '0000950170-23-011328.txt', '0000950170-23-011354.txt', '0000950170-23-011355.txt', '0000950170-23-011358.txt', '0000950170-23-011364.txt', '00009501

### 2. Extract the raw risk factor text

In [4]:
def remove_tags(text):
    # replace the tag included by <> to a \ 
    pattern1 = r'<[^>]*>'
    cleanText = re.sub(pattern1, "|", text)
    
    # replace the multiple continued \ to a single \
    cleanText = re.sub(r'\s*\|+\s*', '|', cleanText)
    cleanText = re.sub(r'\|+', '|', cleanText)
    
    # replace the unicode
    cleanText = html.unescape(cleanText)
    
    # remove the line break
    cleanText = cleanText.replace('\n', '')
    # remove the pure numbers text between \
    cleanText = re.sub(r'\|(\d+)\|', '|', cleanText)
    # remove the unrecognized string
    cleanText = re.sub(r'|\xa0\|', '', cleanText)
    cleanText = re.sub(r'|\xa0|', '', cleanText)
    return cleanText

In [5]:
def separate_main_body(text):
    lastContentInTC_lst = [">form 10-k summary<", ">item 16. form 10-k summary<", r">exhibits, financial statement schedules<", 
                           r">exhibits and financial statement schedules<", r"exhibits and financial statement schedules <",
                           ">exhibits<"]
    start_idx = -1
    for content in lastContentInTC_lst:
        if content in text.lower():
            start_idx = text.lower().find(content)
            break
            
    if start_idx == -1:
        return False
    
    mainBody = text[start_idx:]
    mainBody = html.unescape(mainBody)
    return mainBody

In [6]:
def extract_risk_factors(mainBody, fileName):
    # match the risk factor
    keywordMatch = re.findall(r">item 1a.\s+risk factors\.?\<", mainBody.lower())
    
    # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
    if not keywordMatch:
        keyword = seperated_keyword_rematch(mainBody)
        
    # if match the pattern successfully, just keep the keyword of risk factor
    else:
        keyword = keywordMatch[0]
    
    # check if match successfully
    if not keyword:
        return False
    keywordCheck_lst = ["item", r"item 1a\.?", "risk", "factors", "risk factors"]
    if keyword in keywordCheck_lst:
        print("Warning: It is better to check whether the extraction is correct manually for file", {fileName})
    
    # match the next section of the risk factor
    nextSectionPattern_lst = [r">item 1b.\s+unresolved staff comments\.?\<", r">item 2.\s+description of property\.?\<"]
    for pattern in nextSectionPattern_lst:
        nextSectionMatch = re.findall(pattern, mainBody.lower())
        
        # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
        if not nextSectionMatch:
            nextSection = seperated_next_section_rematch(mainBody, pattern)
            # if re-match successfully, jump out of the loop
            if nextSection:
                break
        
        # if match the pattern successfully, just keep the next section keyword and jump out of the loop
        else:
            nextSection = nextSectionMatch[0]
            break
    
    # check if match successfully
    if not nextSection:
        return False
    
    # if match successfully, extract the risk factor
    start_idx = mainBody.lower().find(keyword) # start index is the position of the keyword occured in the main body
    end_idx = mainBody.lower().find(nextSection) # end index is the position of the next section begin in the main body
    riskFactors = mainBody[start_idx: end_idx]
    return riskFactors

In [7]:
def seperated_keyword_rematch(mainBody):
    cleanedMainBody = remove_tags(mainBody)
    keywordMatch = re.findall(r"i\|?t\|?e\|?m\|?\s+\|?1\|?a\|?.\|?\s+\|?r\|?i\|?s\|?k\|?\s+\|?f\|?a\|?c\|?t\|?o\|?r\|?s\.?", cleanedMainBody.lower())
    
    # check if re-match the keyword of risk factor successfully
    if not keywordMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the risk factor
    reMatchedKeyword = max(keywordMatch[0].split('|'), key=len)
    return reMatchedKeyword
    
def seperated_next_section_rematch(mainBody, nextSectionPattern):
    # cleanse the main body text
    cleanedMainBody = remove_tags(mainBody)
    
    # edit the pattern 
    edittedPattern = nextSectionPattern.replace(r">", "").replace(r"\s+", " ").replace(r"\.?\<", "")
    edittedPattern = "\|?".join(edittedPattern)
    edittedPattern = r"{}\.?".format(edittedPattern)
    pattern = edittedPattern.replace(" ", "\s?+")
    nextSectionMatch = re.findall(pattern, cleanedMainBody.lower())
    
    # check if re-match the keyword of next section successfully
    if not nextSectionMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the next section 
    reMatchedNextSection = max(nextSectionMatch[0].split('|'), key=len)
    return reMatchedNextSection

In [8]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Life Sciences Industry/Industry_2834/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000806592-22-000024.txt'} because of the error in seperating main body.
The extraction is failed for {'0000849146-22-000058.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-22-012521.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-010599.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-011138.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-011139.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001005286-22-000047.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001096906-22-002384.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001104659-22-041326.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000110465

The extraction is failed for {'0001493152-23-010431.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001516551-23-000038.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001520138-22-000437.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001539497-22-000793.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-22-013680.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-22-013940.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-22-013954.txt'} because of the error in seperating main body.
The extraction is failed for {'0001558370-22-014652.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001558370-23-004864.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000155837

### 3. Extract the subjects from raw risk factors text

In [9]:
from bs4 import BeautifulSoup

def extract_subjects(riskFactors, subjectFormat):
    soup = BeautifulSoup(riskFactors, 'html.parser')
    subject_lst = []
    for span in soup.find_all('span'):
        style = span.get('style')
        if style and subjectFormat in style:
            subject_lst.append(span.get_text())
    return subject_lst

In [10]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Life Sciences Industry/Industry_2834/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [11]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Life Sciences Industry/Industry_2834/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001062993-23-008012.txt'}
The subject fail to extract for {'0001140361-23-014526.txt'}
The subject fail to extract for {'0001140361-23-014868.txt'}
The subject fail to extract for {'0001213900-22-027311.txt'}
The subject fail to extract for {'0001213900-23-024260.txt'}
The subject fail to extract for {'0001213900-23-024310.txt'}
The subject fail to extract for {'0001213900-23-024571.txt'}
The subject fail to extract for {'0001213900-23-024710.txt'}
The subject fail to extract for {'0001213900-23-024901.txt'}
The subject fail to extract for {'0001213900-23-025455.txt'}
The subject fail to extract for {'0001213900-23-025554.txt'}
The subject fail to extract for {'0001213900-23-025597.txt'}
The subject fail to extract for {'0001214659-23-004512.txt'}
The subject fail to extract for {'0001410578-22-000947.txt'}
The subject fail to extract for {'0001437749-22-022334.txt'}
The subject fail to extract for {'0001437749-23-008847.txt'}
The subject fail to extr

---

## 2.2.2 Industry 2836

### 1. Read in the files and store in a dictionary

In [13]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Life Sciences Industry/Industry_2836/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

### 2. Extract the raw risk factor text

In [14]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Life Sciences Industry/Industry_2836/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000950170-23-002792.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-008166.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-008517.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-008764.txt'} because of the error in seperating main body.
The extraction is failed for {'0000950170-23-008835.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-009516.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-009527.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-009551.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-010976.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000095017

### 3. Extract the subjects from raw risk factors text

In [15]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Life Sciences Industry/Industry_2836/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [16]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Life Sciences Industry/Industry_2836/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001171843-23-001691.txt'}
The subject fail to extract for {'0001213900-22-057677.txt'}
The subject fail to extract for {'0001213900-23-024122.txt'}
The subject fail to extract for {'0001213900-23-024687.txt'}
The subject fail to extract for {'0001213900-23-024935.txt'}
The subject fail to extract for {'0001477932-23-002090.txt'}
The subject fail to extract for {'0001477932-23-002123.txt'}
The subject fail to extract for {'0001654954-23-003905.txt'}
The subject fail to extract for {'0001683168-22-005189.txt'}
